<a href="https://colab.research.google.com/github/Hlompho-Dash/first_try_of_fastai/blob/master/02_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Mounted at /content/gdrive
------------------------------------------------------------------


In [2]:
print(os.getcwd())

/content/gdrive/MyDrive/first_try_of_fastai


In [3]:
#export

from exp.nb_01 import *

def get_data():
  path2 = "/content/gdrive/MyDrive/first_try_of_fastai/mnist (1).pkl.gz"
  with gzip.open(path2, "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding = "latin-1")
  return map(tensor, (x_train,y_train,x_valid,y_valid))
def normalize(x, m, s): return (x-m)/s

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [6]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [7]:
x_train = normalize(x_train, train_mean, train_std)

x_valid = normalize(x_valid, train_mean, train_std)

In [10]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(3.8966e-08), tensor(1.))

In [11]:
#export

def test_near_zero(a, tol=1e-3):  assert a.abs()<tol, f"Near zero: {a}"

In [12]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [13]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

**Basic architecture**

In [14]:
nh = 50

In [16]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh )
b2 = torch.zeros(1)

In [17]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [18]:
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [24]:
def lin(x, w, b): return x@w + b

In [26]:
t = lin(x_valid, w1, b1)
t.mean(),t.std()

(tensor(-0.0729), tensor(0.9993))

In [27]:
def relu(x): return x.clamp_min(0.)

In [28]:
t = relu(lin(x_valid, w1, b1))

In [29]:
t.mean(),t.std()

(tensor(0.3611), tensor(0.5755))

**  FIXING THE VANISHING GRADIENT PROBLEM**

In [30]:
w1 = torch.rand(m,nh)*math.sqrt(2/m)
w1.mean(),w1.std()

(tensor(0.0254), tensor(0.0146))

In [31]:
t= relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.9981), tensor(1.6849))

In [32]:
 #export

 from torch.nn import init

In [33]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode = "fan_out")

t = relu(lin(x_valid, w1, b1))

In [34]:
init.kaiming_normal_??

In [35]:
w1.mean(),w1.std()

(tensor(-0.0002), tensor(0.0506))

In [36]:
t.mean(),t.std()

(tensor(0.5823), tensor(0.8165))

In [37]:
w1.shape

torch.Size([784, 50])

In [38]:
import torch.nn

In [42]:
torch.nn.Linear.forward??

In [44]:
torch.nn.functional.linear??

In [45]:
torch.nn.Conv2d??

In [46]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [49]:
def relu(x): return x.clamp_min(0.) - 0.5

In [50]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t.std()

(tensor(0.4475), tensor(0.8165))

In [51]:
def model(xb):
  l1 = lin(xb, w1, b1)
  l2 = relu(l1)
  l3 = lin(l2, w2, b2)

  return l3

In [53]:
%timeit -n 10 _= model(x_valid)

10 loops, best of 3: 23.2 ms per loop
